#PolitoSbobinatore

# Insert Your Polito Credentials and Course Information

Before running the script, fill in your PoliTo credentials and the course details:


In [ ]:
#INSERT HERE YOUR DATA
USERNAME = "s******" #POLITO USERNAME
PASSWORD =  "*******" #POLITO PASSWORD
COURSE_TITLE = "*********"
LECTURE_TITLE = "********"


### Installation of Dependencies and Environment Setup
In the next block, we will install the necessary dependencies and set up the environment to run the subsequent code. This includes installing libraries such as Whisper, MoviePy, Transformers, Selenium, and others, as well as configuring the Chromium browser.

In [ ]:
import time
total_time = 0
start_time = time.time()

!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg
!pip install moviepy
!pip install --upgrade transformers torch
!pip install torch
!pip install transformers
!pip install selenium webdriver_manager requests
!apt-get update -y
!apt-get install -y chromium-browser
!apt-get install -y chromedriver
!apt install chromium-chromedriver

end_time = time.time()
total_time = total_time + end_time - start_time

# Automated Login and Video Download from Virtual Classroom

This script uses Selenium to automate the login to the Politecnico di Torino portal, navigate through the platform, and download the lecture video from the Virtual Classroom. The process includes:

1. Automatic login to the PoliTo portal with credentials.
2. Navigation to the specified course and desired lecture.
3. Extraction of the video URL from the lecture page.
4. Downloading the video as an MP4 file to the local system.

The browser is run in headless mode (without a graphical interface) for faster processing and without the need for manual interaction.


In [ ]:
start_time = time.time()
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import requests

# Settings to use Chromium in headless mode
chrome_options = Options()
chrome_options.add_argument('--headless')  # Headless mode
chrome_options.add_argument('--no-sandbox')  # Necessary for running in a container environment
chrome_options.add_argument('--disable-dev-shm-usage')  # Necessary to avoid memory issues

# Configure the Chromium driver
driver = webdriver.Chrome(options=chrome_options)

# URL and credentials
URL = "https://idp.polito.it/home"
USERNAME_FIELD_ID = "username"
PASSWORD_FIELD_ID = "password"
LOGIN_BUTTON_CSS_SELECTOR = "button.login.button"

# Maximize the window
driver.maximize_window()

try:
    # Open the login page
    driver.get(URL)

    # Find the username field and enter the value
    username_box = driver.find_element(By.ID, USERNAME_FIELD_ID)
    username_box.send_keys(USERNAME)

    # Find the password field and enter the value
    password_box = driver.find_element(By.ID, PASSWORD_FIELD_ID)
    password_box.send_keys(PASSWORD)

    # Find and click the login button
    login_button = driver.find_element(By.CSS_SELECTOR, LOGIN_BUTTON_CSS_SELECTOR)
    login_button.click()

    # Wait 10 seconds for the page to load
    time.sleep(10)

    if(driver.title == "Portale della Didattica - Studente"):
      print("Login Completed ")
    else:
      print("Login Failed")
      driver.quit()

    if "MyPoli" in driver.title:
        link_portale = driver.get("https://didattica.polito.it/pls/portal30/sviluppo.pagina_studente_2016.main")


    # Print the page title for verification
    print("Page title:", driver.title)

    # Navigate to the specified course using the course title
    link_Course = driver.find_element(By.LINK_TEXT, COURSE_TITLE)
    link_Course.click()

    # Click the link for the "Virtual classroom"
    link_virtualClass = driver.find_element(By.LINK_TEXT, "Virtual classroom")
    link_virtualClass.click()

    # Find and click the link for the specific lecture "Efficient fine-tuning, inference"
    link_vc = driver.find_element(By.XPATH, f"//a[text()='{LECTURE_TITLE}']")
    link_vc.click()

    # Wait 10 seconds for the video to load
    time.sleep(10)

    # Find the video element using the class 'video-js' and get the video URL
    video_element = driver.find_element(By.CLASS_NAME, "video-js")
    video_url = video_element.find_element(By.TAG_NAME, "source").get_attribute("src")

    # Download the video using the obtained URL
    print(f"Video URL found: {video_url}")
    video_response = requests.get(video_url)

    # Save the video to disk with the name "video.mp4"
    with open("video.mp4", "wb") as video_file:
        video_file.write(video_response.content)
    print("Video downloaded successfully!")

    # Print the page title for verification
    print("Page title:", driver.title)

finally:
    # Wait 10 seconds to see the result
    time.sleep(10)
    # Close the browser
    driver.quit()
end_time = time.time()
total_time = total_time + end_time - start_time

# Extracting Audio from Video and Transcription with Whisper

This script uses `moviepy` to extract audio from a video file and save it as an MP3 file, then uses OpenAI's Whisper model for potential transcription. The process includes:

1. Loading the video file in MP4 format.
2. Extracting the audio and saving it as an MP3 file.
3. Optionally, using Whisper for audio transcription (not included in this snippet but could be added).
   
The result is an MP3 file that contains the audio from the video, ready for further processing or transcription.


In [ ]:
from moviepy.editor import *
import whisper
import time

start_time = time.time()

# Load the Whisper model
model = whisper.load_model("small")  # Load the model on the appropriate device

# Path to the MP4 file
input_file = "video.mp4"
# Name of the output MP3 file
output_file = "audio.mp3"

# Load the video
video = VideoFileClip(input_file)

# Extract the audio and save it as MP3
video.audio.write_audiofile(output_file)

print(f"Conversion completed! File saved as: {output_file}")

# End the timer and calculate the total time
end_time = time.time()
total_time = end_time - start_time
print(f"Total operation time: {total_time:.4f} seconds")


# Audio Transcription with Whisper

This script uses OpenAI's Whisper model to transcribe audio from an MP3 file into text. The process includes:

1. Loading the audio file in MP3 format.
2. Transcribing the audio using the Whisper model.
3. Saving the transcription as a text file.

The result is a text file containing the full transcription of the audio, ready for review or further processing.


In [ ]:
start_time = time.time()
# Path to the MP3 file
input_audio = "audio.mp3"
# Name of the output file
output_text = "trascription.txt"

# Transcribe the audio
result = model.transcribe(input_audio)

# Save the transcription to a text file
with open(output_text, "w", encoding="utf-8") as file:
    file.write(result["text"])

end_time = time.time()
total_time = total_time + end_time - start_time

# Text Summarization with LLama 3.2

This script uses the LLama 3.2 model from Hugging Face's Transformers library to generate a summary of a given transcription. The process includes:

1. Reading the transcription text from a file.
2. Loading the LLama model for summarization (`meta-llama/Llama-3.2-3B-Instruct`).
3. Preparing the content for summarization.

This script sets up the model and tokenizer, ready to summarize the transcription text into a concise version.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
import torch
from huggingface_hub import login
import time



# Caricamento modello con ottimizzazioni
model_id = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True
)

# Verifica dispositivo
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

print("Model loaded successfully.")

## Llama 3.2 3B Lecture Summarization Script Documentation

This script generates a summary for a large transcription using the LLaMA 3.2 family model, which supports an extended context window. The prompt is structured to ensure an academic and coherent summary by enforcing a continuous prose style, organized sections, and clear transitions. It maintains an academic tone, limits paragraph length for readability, and excludes irrelevant details. Mathematical content is formatted with LaTeX, while conceptual continuity is reinforced through structured derivations and applied examples. These elements contribute to producing high-quality, logically structured summaries suitable for lecture material.



In [ ]:

# Lettura file
file_path = "trascription.txt"
with open(file_path, "r", encoding="utf-8") as file:
    transcription_text = file.read()

# Costruzione prompt ottimizzato
system_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a technical writer creating comprehensive lecture summaries. Structure your output in continuous prose with these sections:

**Formatting Rules:**
1. Never use bullet points or numbered lists
2. Use subheadings with ## notation
3. Keep paragraphs under 10 sentences
4. Maintain third-person academic tone
5. Explicitly mention when transitioning between topics
6. Include pratical examples
7. Keep a connections between the different parts
8. Exclude useless information
9. Use transitional phrases:
   - 'Building upon the previous discussion of X...'
   - 'This finding naturally leads to consideration of Y...'
10. Create conceptual bridges between sections using:
   - Mathematical continuity (derive Equation 2 from Equation 1)
   - Applied case studies demonstrating theoretical principles
11. Enclose equations in \( \) for inline and \[ \] for display
12. Use LaTeX environments for multi-line proofs:
   \[
   \begin{align}
       x &= \frac{-b \pm \sqrt{b^2 - 4ac}}{2a} \\
       \frac{dy}{dx} &= 3x^2 + 2x
   \end{align}
   \]

<|eot_id|>
"""


user_prompt = f"""<|start_header_id|>user<|end_header_id|>
{transcription_text}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
Output:
"""

# Tokenizzazione con gestione lunghezza
inputs = tokenizer(
    system_prompt + user_prompt,
    return_tensors="pt",
).to(device)

# Generazione con parametri ottimizzati
start_time = time.time()

with torch.inference_mode():
    outputs = model.generate(
        **inputs,
        max_new_tokens=2048,
        min_new_tokens=512,
        temperature=0.6,
        top_p=0.9,
        repetition_penalty=1.1,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

# Decodifica e pulizia output
full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
summary = full_text.split("Output:")[1].strip()

# Salvataggio risultati
total_time = time.time() - start_time
with open("final_summary.txt", "w", encoding="utf-8") as f:
    f.write(f"Lecture Summary:\n{summary}\n")
    f.write(f"\nGeneration time: {total_time:.2f} seconds")

print(f"Riassunto completato in {total_time:.2f} secondi")
